In [21]:
import os
import re

In [22]:
TOTAL = False

In [23]:
directory = "../data"

if not TOTAL:
    directory = "../pre/sample"

data_list = os.listdir(directory)
data_list = list(data_list)

In [33]:
word_count = {}

for file in data_list:
    with open(f'{directory}/{file}', 'r') as f:
        text = f.read()
        text = text.lower()
        text = re.sub(r'[^\w0-9- ]+', '', text, flags=re.UNICODE)

        text = [x.strip() for x in text.split() if len(x) > 0]

        for word in text:
            if word in word_count:
                word_count[word] += 1
            else:
                word_count[word] = 1

In [ ]:
print(len(word_count.keys()))